In [1]:
import pandas as pd
import numpy as np

def load_data(csv_file):
    data = pd.read_csv(csv_file)
    data_col_list = data.columns.to_list()
    x_col_list = data_col_list[:-1]
    print(f"Independent Values;\n{x_col_list}")

    y_col = data_col_list[-1]
    #Extract the independent variables (features) into matrix X
    X = data[x_col_list].values

    #Extract the dependent variable into vector Y
    Y = data[y_col].values
    print(f"dependent Values;\n{y_col}")
    return X, Y


csv_file = 'dataset/Student_Performance_preprocess_normalized_minmax.csv'
X, Y = load_data(csv_file)

Independent Values;
['Hours Studied', 'Previous Scores', 'Extracurricular Activities', 'Sleep Hours', 'Sample Question Papers Practiced']
dependent Values;
Performance Index


In [2]:
import autosklearn.regression

from sklearn.model_selection import train_test_split

# Load your dataset (X and Y)
# Replace this with your dataset loading code
# X, Y = load_your_data()

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create an instance of Auto-sklearn regressor
automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=60,  # Time limit in seconds
    per_run_time_limit=60,        # Time limit for each model
    ensemble_size=1,               # Number of models to ensemble
    initial_configurations_via_metalearning=0,
    seed=42                        # Random seed for reproducibility
)

# Fit Auto-sklearn to your training data
automl.fit(X_train, Y_train)

# Get predictions on the test data
Y_pred = automl.predict(X_test)

# Evaluate the model
from sklearn.metrics import r2_score, mean_squared_error

r2 = r2_score(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)

print("R2 Score:", r2)
print("Mean Squared Error:", mse)

# Print the best model found by Auto-sklearn
print("Best Model:", automl.show_models())


/tmp/ipykernel_333518/2873297455.py:13: DeprecationWarning: `ensemble_size` has been deprecated, please use `ensemble_kwargs = {'ensemble_size': 1}`. Inserting `ensemble_size` into `ensemble_kwargs` for now. `ensemble_size` will be removed in auto-sklearn 0.16.
  automl = autosklearn.regression.AutoSklearnRegressor(


[WARNING] [2023-09-17 02:19:37,284:Client-AutoML(42):bd118678-54d6-11ee-96ce-c7e84a0062f3] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (59.514654)
[WARNING] [2023-09-17 02:19:37,284:Client-AutoML(42):bd118678-54d6-11ee-96ce-c7e84a0062f3] Capping the per_run_time_limit to 29.0 to have time for a least 2 models in each process.
R2 Score: 0.9889785125825429
Mean Squared Error: 4.084399193090869
Best Model: {8: {'model_id': 8, 'rank': 1, 'cost': 0.01144159542293588, 'ensemble_weight': 1.0, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f3141aae640>, 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f3141b65f10>, 'regressor': <autosklearn.pipeline.components.regression.RegressorChoice object at 0x7f3141b65160>, 'sklearn_regressor': ARDRegression(alpha_1=0.000902745247612653, alpha

In [4]:
print(X_test)

[[0.5        0.49152542 0.         0.8        0.22222222]
 [0.125      0.10169492 1.         0.         0.88888889]
 [0.75       0.27118644 1.         0.6        0.55555556]
 ...
 [0.125      0.         1.         0.6        0.22222222]
 [0.25       0.72881356 0.         0.6        0.55555556]
 [0.5        0.33898305 0.         0.8        0.44444444]]


In [5]:
print(automl.predict([np.array([0.500,    0.203390,	1.0,	0.2,	0.22222])]))

[36.59601593]


In [7]:
import joblib

model_save_path = 'models/automl_Performance_Linear_Regression.pkl'  # Replace with your desired file path

# Save the trained model using joblib
joblib.dump(automl, model_save_path)

print("Model saved to:", model_save_path)


Model saved to: models/automl_Performance_Linear_Regression.pkl


In [1]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="models/automl_Performance_Linear_Regression.pkl",
    path_in_repo="model.pkl",
    repo_id="sharoz/automl_Student_Marks",
    repo_type="model",
)

automl_Performance_Linear_Regression.pkl:   0%|          | 0.00/377k [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

'https://huggingface.co/sharoz/automl_Student_Marks/blob/main/model.pkl'